In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [1,011 kB]
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:12 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:13 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True, inferSchema=True)
df.show()

+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|               date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+-------------------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08 00:00:00|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13 00:00:00|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12 00:00:00|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16 00:00:00|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08 00:00:00|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa0052

In [ ]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('sales')

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
avg_4_bedrooms = spark.sql("""
SELECT 
  YEAR(date) AS Year,
  round(avg(price), 2) AS Average_price
FROM sales 
WHERE bedrooms = 4
GROUP BY Year
ORDER BY Year DESC

""")
avg_4_bedrooms.show()


+----+-------------+
|Year|Average_price|
+----+-------------+
|2022|    296363.88|
|2021|    301819.44|
|2020|    298353.78|
|2019|     300263.7|
+----+-------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_3_bedrooms_bathrooms = spark.sql("""
SELECT
  date_built AS Year,
  ROUND(AVG(price), 2) AS Average_price
FROM sales
WHERE bedrooms = 3
and bathrooms = 3
GROUP BY Year
ORDER BY Year DESC
""")
avg_3_bedrooms_bathrooms.show()



+----+-------------+
|Year|Average_price|
+----+-------------+
|2017|    292676.79|
|2016|    290555.07|
|2015|     288770.3|
|2014|    290852.27|
|2013|    295962.27|
|2012|    293683.19|
|2011|    291117.47|
|2010|    292859.62|
+----+-------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
avg_2_floors = spark.sql("""
SELECT
  date_built AS Year,
  ROUND(AVG(price), 2) AS Average_price
FROM sales
WHERE bedrooms = 3
and bathrooms = 3
and floors = 2
and sqft_living >= 2000
GROUP BY Year
ORDER BY Year DESC
""")
avg_2_floors.show()


+----+-------------+
|Year|Average_price|
+----+-------------+
|2017|    280317.58|
|2016|     293965.1|
|2015|    297609.97|
|2014|    298264.72|
|2013|    303676.79|
|2012|    307539.97|
|2011|    276553.81|
|2010|    285010.22|
+----+-------------+



In [23]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

avg_price_per_view = spark.sql("""
SELECT
  view as Number_of_Views,
  ROUND(AVG(price), 2) AS Average_price
FROM sales
WHERE price  >= 350000
GROUP BY Number_of_Views
ORDER BY Number_of_Views DESC
""")
avg_price_per_view.show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------------+-------------+
|Number_of_Views|Average_price|
+---------------+-------------+
|            100|    1026669.5|
|             99|   1061201.42|
|             98|   1053739.33|
|             97|   1129040.15|
|             96|   1017815.92|
|             95|    1054325.6|
|             94|    1033536.2|
|             93|   1026006.06|
|             92|    970402.55|
|             91|   1137372.73|
|             90|   1062654.16|
|             89|   1107839.15|
|             88|   1031719.35|
|             87|    1072285.2|
|             86|   1070444.25|
|             85|   1056336.74|
|             84|   1117233.13|
|             83|   1033965.93|
|             82|    1063498.0|
|             81|   1053472.79|
+---------------+-------------+
only showing top 20 rows

--- 1.0969228744506836 seconds ---


In [24]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table sales")

DataFrame[]

In [25]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales')

True

In [26]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
avg_price_cached = spark.sql("""
SELECT
  view as Number_of_Views,
  ROUND(AVG(price), 2) AS Average_price
FROM sales
WHERE price  >= 350000
GROUP BY Number_of_Views
ORDER BY Number_of_Views DESC
""")
avg_price_cached.show()


print("--- %s seconds ---" % (time.time() - start_time))


+---------------+-------------+
|Number_of_Views|Average_price|
+---------------+-------------+
|            100|    1026669.5|
|             99|   1061201.42|
|             98|   1053739.33|
|             97|   1129040.15|
|             96|   1017815.92|
|             95|    1054325.6|
|             94|    1033536.2|
|             93|   1026006.06|
|             92|    970402.55|
|             91|   1137372.73|
|             90|   1062654.16|
|             89|   1107839.15|
|             88|   1031719.35|
|             87|    1072285.2|
|             86|   1070444.25|
|             85|   1056336.74|
|             84|   1117233.13|
|             83|   1033965.93|
|             82|    1063498.0|
|             81|   1053472.79|
+---------------+-------------+
only showing top 20 rows

--- 0.6823029518127441 seconds ---


In [28]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
df.write.partitionBy('date_built').parquet('home_sales',mode='overwrite')

In [30]:
# 11. Read the parquet formatted data.
df = spark.read.parquet('home_sales')
df.show()

+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|                  id|               date| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|date_built|
+--------------------+-------------------+------+--------+---------+-----------+--------+------+----------+----+----------+
|2ed8d509-7372-46d...|2021-08-06 00:00:00|258710|       3|        3|       1918|    9666|     1|         0|  25|      2015|
|941bad30-eb49-4a7...|2020-05-09 00:00:00|229896|       3|        3|       2197|    8641|     1|         0|   3|      2015|
|c797ca12-52cd-4b1...|2019-06-08 00:00:00|288650|       2|        3|       2100|   10419|     2|         0|   7|      2015|
|0cfe57f3-28c2-472...|2019-10-04 00:00:00|308313|       3|        3|       1960|    9453|     2|         0|   2|      2015|
|d715f295-2fbf-4e9...|2021-05-17 00:00:00|391574|       3|        2|       1635|    8040|     2|         0|  10|      2015|
|a18515a

In [31]:
# 12. Create a temporary table for the parquet data.
df.createOrReplaceTempView('home_sales_table')

In [32]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time = time.time()
avg_price_filtered = spark.sql("""
SELECT
  view as Number_of_Views,
  ROUND(AVG(price), 2) AS Average_price
FROM sales
WHERE price  >= 350000
GROUP BY Number_of_Views
ORDER BY Number_of_Views DESC
""")
avg_price_filtered.show()


print("--- %s seconds ---" % (time.time() - start_time))

+---------------+-------------+
|Number_of_Views|Average_price|
+---------------+-------------+
|            100|    1026669.5|
|             99|   1061201.42|
|             98|   1053739.33|
|             97|   1129040.15|
|             96|   1017815.92|
|             95|    1054325.6|
|             94|    1033536.2|
|             93|   1026006.06|
|             92|    970402.55|
|             91|   1137372.73|
|             90|   1062654.16|
|             89|   1107839.15|
|             88|   1031719.35|
|             87|    1072285.2|
|             86|   1070444.25|
|             85|   1056336.74|
|             84|   1117233.13|
|             83|   1033965.93|
|             82|    1063498.0|
|             81|   1053472.79|
+---------------+-------------+
only showing top 20 rows

--- 0.4756779670715332 seconds ---


In [33]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table sales')

DataFrame[]

In [34]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('sales')


False